# Behavior statistics

This notebook panels for Figure 2 showing examples of the stimuli used in the behavioral trials.

In [ ]:
import sys

sys.path.insert(0, "../scripts")

In [ ]:
from pathlib import Path

import ewave
import matplotlib.pyplot as plt
from dlab import plotting
import graphics_defaults # noqa: F401

In [ ]:
stim_file_path = Path("../datasets/zebf-discrim-noise/stimuli")

In [ ]:
foreground = "9ex2k0dy"
background = "btwmt59w1"
background_levels = [-100, -65, -60, -55, -50, -45, -40, -35, -30, -25, -20]
background_levels = [-100, -60, -50, -40, -30, -20]

In [ ]:
fig, axs = plt.subplots(nrows=len(background_levels), sharex=True, sharey=True, figsize=(1.5, 2.5), dpi=300)

for ax, level in zip(axs, background_levels):
    stim_name = f"{foreground}-30_{background}{level}"
    stim_file = stim_file_path / stim_name
    with ewave.wavfile(stim_file.with_suffix(".wav"), "r") as fp:
        signal, sampling_rate = fp.read(), fp.sampling_rate
    plotting.spectrogram(ax, signal, sampling_rate, frequency_range=(0, 8000), compression=0.01)
    ax.set_yticks([0, 5000], ["0", "5"])
ax.set_xlim(0, 3.0)
for ax in axs[:-1]:
    ax.get_xaxis().set_visible(False)
    ax.set_frame_on(False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
fig.subplots_adjust(hspace=0.05, wspace=0.1)

In [ ]:
fig.savefig("../figures/behavior_example_stims.pdf")